In [21]:
# import libraries
# import boto3, re, sys, math, json, os, sagemaker, urllib.request
# from sagemaker import get_execution_role
import numpy as np                                
import pandas as pd                               
import matplotlib.pyplot as plt                   
from IPython.display import Image                 
from IPython.display import display               
from time import gmtime, strftime                 
# from sagemaker.predictor import csv_serializer   

import pandas as pd
import numpy as np
from pandas.api.types import CategoricalDtype    
 
from sklearn.model_selection import train_test_split
from sklearn import preprocessing
from sklearn.ensemble import RandomForestClassifier
from sklearn.pipeline import make_pipeline
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.externals import joblib 


In [22]:
try:
    df = pd.read_csv('../cc_attach_project.csv',index_col=0, nrows=50000)
    print('Success: Data loaded into dataframe.')
except Exception as e:
    print('Data load error: ',e)


Success: Data loaded into dataframe.


In [23]:

to_drop = ['ENROLLMENT_DATE', 'DATE']
df.drop(to_drop, inplace=True, axis=1)
# df['ENROLLMENT_DATE'] = pd.to_datetime(df['ENROLLMENT_DATE'])
# df['DATE'] = pd.to_datetime(df['DATE'])
df['STAGENAME_PRODUCT'] = df['STAGENAME_PRODUCT'].fillna('not called')
df['INDUSTRY'] = df['INDUSTRY'].fillna('missing')
df['CASH_PERC'] = df['CASH_PERC'].fillna(0.0)  
df['CHECK_PERC'] = df['CHECK_PERC'].fillna(0.0)
df['OTHER_PERC'] = df['OTHER_PERC'].fillna(0.0)
df['ACH_PERC'] = df['ACH_PERC'].fillna(0.0)
df['CC_PERC'] = df['CC_PERC'].fillna(0.0)
df['BILLING_PLAN_ID'] = df['BILLING_PLAN_ID'].astype('int64')
cat_dtype = CategoricalDtype(categories=(df['PLAN_TIER'].unique()), ordered=True)
df['PLAN_TIER'] = df['PLAN_TIER'].astype(cat_dtype) #small > medium?
df['INDUSTRY'] = df['INDUSTRY'].astype('category')
df['ORGSIZE'] = df['ORGSIZE'].abs()
df['ORGSIZE'].replace(0,1, inplace=True) #should I do this?


#convert to all floats
# df = df.apply(pd.to_numeric, errors='ignore', downcast='float')

# df = pd.get_dummies(df)

In [24]:
# for column in df:
#     if df[column].dtype == np.float32 or df[column].dtype == np.int64:
#         df[column] = df[column].astype('float64')
# print(df.dtypes)

In [25]:
df = pd.get_dummies(df)
print(df.shape)

(50000, 98)


In [26]:
y = df['WILL_ATTACH_IN_60']
X = df.drop('WILL_ATTACH_IN_60', axis=1)
X_train, X_test, y_train, y_test = train_test_split(X, y, 
                                                    test_size=0.15, 
                                                    random_state=123, 
                                                    stratify=y)
print(X_train.shape, X_test.shape, y_train.shape, y_test.shape)


(42500, 97) (7500, 97) (42500,) (7500,)


In [30]:
# 5. Declare data preprocessing steps
# pipeline = make_pipeline(preprocessing.StandardScaler(), 
#                          RandomForestClassifier(n_estimators=100, warm_start=True))
# 6. Declare hyperparameters to tune
# hyperparameters = { 'randomforestclassifier__max_features' : ['auto', 'sqrt', 'log2'],
#                   'randomforestclassifier__max_depth': [None, 5, 3, 1]}
 
# 7. Tune model using cross-validation pipeline
# clf = GridSearchCV(pipeline, hyperparameters, cv=10, refit=True,return_train_score=True)
clf = RandomForestClassifier(n_estimators=200, max_depth=None, max_features='sqrt', n_jobs = -1, oob_score = True,
                              random_state=1)

clf.fit(X_train, y_train)
print(clf.score(X_test,y_test))

# print(clf.feature_importances_)


0.9977333333333334
1.0


In [28]:
# X_trains, y_trains = np.array_split(X_train, indices_or_sections=20), np.array_split(y_train, indices_or_sections=20)    

# for X_train_section, y_train_section in zip(X_trains, y_trains):
#     print("TRAIN:", X_train_section.shape, "TEST:", y_train_section.shape)
#     clf.fit(X_train_section, y_train_section)
# print(clf.score(X_test,y_test))
       


In [29]:
print(clf.best_params_)
# print(clf.feature_importances_)
results = clf.cv_results_
print(clf.best_score_)
print(clf.best_estimator_)
print(results['mean_train_score'])
print(results['mean_test_score'])
# joblib.dump(clf, 'rf_regressor.pkl')


AttributeError: 'RandomForestClassifier' object has no attribute 'best_params_'